# Домашнее задание №15. Keras

## Задание 1
1. Самостоятельно выбранными средствами (opencv, pillow (PIL), …) сгенерировать по 820 картинок размером 100х100 пикселей (px) для каждой из цифр: 0, 1, 3, 8 следующим образом (800 – тренировочная выборка, 20 – тестовая выборка № 1):
* фон картинки белый,
* цифра: ширина – 20 px, высота – 50 px, цвет линии – черный, цифра целиком помещается в картинку, цифра находится в случайном месте на картинке, 
* на изображении цифра расположена так, что ее вертикальная ось параллельна оси ординат (вертикальное положение) или оси абсцисс (горизонтальное положение),
* тренировочная выборка содержит 400 изображений каждой цифры в горизонтальном положении и 400 изображений каждой цифры в вертикальном положении,
* тестовая выборка содержит 10 изображений каждой цифры в горизонтальном положении и 10 изображений каждой цифры в вертикальном положении,

2. Создать новые тестовые картинки, полученные путем добавления черных пикселей (шум) в случайно выбранные места сгенерированных тестовых картинок:
* 20 пикселей (тестовая выборка № 2),
* 50 пикселей (тестовая выборка № 3), 
* 100 пикселей (тестовая выборка № 4),
* 200 пикселей (тестовая выборка № 5).

In [10]:
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input
from tensorflow.keras.utils import to_categorical
import pandas as pd

In [11]:
# 1. Генерация изображений
def generate_images():
    # Параметры изображения
    image_size = (100, 100)
    digit_size = (20, 50)
    background_color = (255, 255, 255)  # белый
    digit_color = (0, 0, 0)             # черный
    digits = ['0', '1', '3', '8']

    # Создание папок
    def create_folders():
        folders = [
            'train/horizontal', 'train/vertical',
            'test1/horizontal', 'test1/vertical',
            'test2/horizontal', 'test2/vertical',
            'test3/horizontal', 'test3/vertical',
            'test4/horizontal', 'test4/vertical',
            'test5/horizontal', 'test5/vertical'
        ]
        for folder in folders:
            for digit in digits:
                os.makedirs(os.path.join(folder, digit), exist_ok=True)

    # Генерация одной цифры
    def generate_digit(digit, orientation, path, count):
        img = Image.new('RGB', image_size, background_color)
        draw = ImageDraw.Draw(img)
        
        try:
            font = ImageFont.truetype("arial.ttf", size=max(digit_size))
        except:
            font = ImageFont.load_default()
            font_size = 1
            while True:
                test_font = ImageFont.load_default()
                left, top, right, bottom = draw.textbbox((0, 0), digit, font=test_font)
                w, h = right - left, bottom - top
                if w > digit_size[0] or h > digit_size[1]:
                    break
                font = test_font
                font_size += 1
        
        left, top, right, bottom = draw.textbbox((0, 0), digit, font=font)
        digit_width, digit_height = right - left, bottom - top
        
        if orientation == 'horizontal':
            max_x = image_size[0] - digit_width
            max_y = image_size[1] - digit_height
            pos_x = random.randint(0, max_x)
            pos_y = random.randint(0, max_y)
            position = (pos_x, pos_y)
        else:
            img = img.rotate(90, expand=True)
            draw = ImageDraw.Draw(img)
            max_x = image_size[1] - digit_width
            max_y = image_size[0] - digit_height
            pos_x = random.randint(0, max_x)
            pos_y = random.randint(0, max_y)
            position = (pos_x, pos_y)
        
        draw.text(position, digit, fill=digit_color, font=font)
        
        if orientation == 'vertical':
            img = img.rotate(-90, expand=True)
        
        img.save(os.path.join(path, orientation, digit, f'{digit}_{count}.png'))

    # Добавление шума
    def add_noise(image_path, output_path, noise_pixels):
        img = Image.open(image_path)
        pixels = img.load()
        width, height = img.size
        
        for _ in range(noise_pixels):
            x = random.randint(0, width - 1)
            y = random.randint(0, height - 1)
            pixels[x, y] = digit_color
        
        img.save(output_path)

    # Основная генерация
    create_folders()
    
    # Тренировочные данные (800 на цифру)
    for digit in digits:
        for i in range(400):
            generate_digit(digit, 'horizontal', 'train', i)
            generate_digit(digit, 'vertical', 'train', i)
    
    # Тестовые данные (20 на цифру)
    for digit in digits:
        for i in range(10):
            generate_digit(digit, 'horizontal', 'test1', i)
            generate_digit(digit, 'vertical', 'test1', i)
    
    # Тестовые выборки с шумом
    for digit in digits:
        for orientation in ['horizontal', 'vertical']:
            for i in range(10):
                input_path = os.path.join('test1', orientation, digit, f'{digit}_{i}.png')
                
                for noise, test_num in [(20, 'test2'), (50, 'test3'), (100, 'test4'), (200, 'test5')]:
                    output_path = os.path.join(test_num, orientation, digit, f'{digit}_{i}.png')
                    add_noise(input_path, output_path, noise)


generate_images()

# Задание №2

Не используя предобученные модели (сети), модифицировать скрипт задачи «Dogs vs Cats» (с семинара) или написать свою нейронную сеть на keras такую, что:

1) На вход подается тренировочное множество: по 800 картинок каждой цифры.
2) Из тренировочного множества выделяется часть картинок (10-20%), на валидационное множество, в котором должны присутствовать цифры в вертикальном и горизонтальном положении.
3) Протестировать адекватность модели на всех тестовых выборках № 1, № 2, № 3, № 4, № 5, фиксируя при этом точность (accuracy) классификации.
4) Повторить пункты 1)–3), изменив объем тренировочной выборки до 600, 400, 200, 100 картинок каждой цифры.

<i> Могут пригодиться <tt>Dense</tt>, <tt>Conv2D</tt>, <tt>MaxPooling2D</tt>, <tt>Flatten</tt>.</i>

Результаты оформить в виде таблицы со столбцами: размер тренировочной выборки, количество шумовых пикселей, точность (accuracy) классификации.

In [ ]:
def run_experiments():
    # Основные параметры данных
    digits = ['0', '1', '3', '8']  # Цифры для классификации
    image_size = (100, 100)        # Размер изображений
    input_shape = (100, 100, 1)    # Формат входных данных для сети (с каналом grayscale)
    num_classes = 4                # Количество классов классификации
    epochs = 15                    # Количество эпох обучения
    batch_size = 32                # Размер батча

    # Загрузка изображений
    def load_images(folder, digit, orientation):
        images = []
        labels = []
        path = os.path.join(folder, orientation, digit)
        
        # Проверка существования папки
        if not os.path.exists(path):
            return [], []
            
        # Чтение всех изображений в папке
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img = Image.open(img_path).convert('L')  # Конвертация в grayscale
            img = np.array(img) / 255.0              # Нормализация [0, 1]
            images.append(img)
            labels.append(digits.index(digit))       # Метка как индекс цифры
        return images, labels

    # Загрузка всех данных
    def load_all_data(folder):
        all_images = []
        all_labels = []
        # Для каждой цифры и ориентации загружаем изображения
        for digit in digits:
            for orientation in ['horizontal', 'vertical']:
                images, labels = load_images(folder, digit, orientation)
                all_images.extend(images)
                all_labels.extend(labels)
        return np.array(all_images), np.array(all_labels)

    # Создание модели
    def create_model():
        model = Sequential([
            # Первый сверточный слой
            Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
            MaxPooling2D(pool_size=(2, 2)),
            # Второй сверточный слой
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            # Преобразование в 1D для полносвязных слоев
            Flatten(),
            # Полносвязный слой
            Dense(128, activation='relu'),
            # Выходной слой с softmax для классификации
            Dense(num_classes, activation='softmax')
        ])
        
        # Компиляция модели
        model.compile(loss='categorical_crossentropy', 
                     optimizer='adam', 
                     metrics=['accuracy'])
        return model

    # Основной эксперимент
    results = []  # Для хранения результатов
    train_sizes = [800, 600, 400, 200, 100]  # Размеры обучающих выборок
    
    for train_size in train_sizes:
        # Загрузка полного тренировочного набора
        X_train_full, y_train_full = load_all_data('train')
        
        # Уменьшение выборки если требуется
        if train_size < 800:
            # Стратифицированное разбиение для сохранения баланса классов
            X_train, _, y_train, _ = train_test_split(
                X_train_full, y_train_full, 
                train_size=train_size*4,  # 4 класса цифр
                stratify=y_train_full)
        else:
            X_train, y_train = X_train_full, y_train_full
        
        # Разделение на тренировочную и валидационную выборки (15% валидации)
        X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, 
            test_size=0.15, 
            stratify=y_train)
        
        # Преобразование меток в one-hot encoding
        y_train = to_categorical(y_train, num_classes)
        y_val = to_categorical(y_val, num_classes)
        
        # Создание и обучение модели
        model = create_model()
        model.fit(
            X_train.reshape(-1, *input_shape),  # Изменение формы данных
            y_train,
            batch_size=batch_size,
            epochs=epochs,
            # verbose=0,  # Без вывода процесса обучения
            validation_data=(X_val.reshape(-1, *input_shape), y_val))
        
        # Тестирование на всех вариантах шума
        for noise_pixels, test_num in [(0, 'test1'), (20, 'test2'), 
                                     (50, 'test3'), (100, 'test4'), 
                                     (200, 'test5')]:
            X_test, y_test = load_all_data(test_num)
            if len(X_test) == 0:  # Пропуск если нет данных
                continue
                
            y_test = to_categorical(y_test, num_classes)
            # Оценка точности на тестовых данных
            loss, accuracy = model.evaluate(
                X_test.reshape(-1, *input_shape), 
                y_test, 
                verbose=0)
            
            # Сохранение результатов
            results.append({
                'Train size': train_size,        # Размер обучающей выборки
                'Noise pixels': noise_pixels,    # Количество шумовых пикселей
                'Accuracy': accuracy             # Точность классификации
            })
    
    # Анализ и сохранение результатов
    df = pd.DataFrame(results)
    print("\nРезультаты:")
    # Сводная таблица точности по разным условиям
    print(df.pivot_table(index='Train size', 
                        columns='Noise pixels', 
                        values='Accuracy'))
    
    # Сохранение в CSV файл
    df.to_csv('experiment_results.csv', index=False)
    return df

In [13]:
results = run_experiments()

Epoch 1/15


c:\Users\murza\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


85/85 ━━━━━━━━━━━━━━━━━━━━ 8s 85ms/step - accuracy: 0.3315 - loss: 1.8122 - val_accuracy: 0.6354 - val_loss: 0.8423
Epoch 2/15
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - accuracy: 0.6964 - loss: 0.7251 - val_accuracy: 0.7271 - val_loss: 0.6484
Epoch 3/15
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - accuracy: 0.8197 - loss: 0.4880 - val_accuracy: 0.7479 - val_loss: 0.5307
Epoch 4/15
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 81ms/step - accuracy: 0.8900 - loss: 0.3202 - val_accuracy: 0.8125 - val_loss: 0.4299
Epoch 5/15
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - accuracy: 0.9562 - loss: 0.1888 - val_accuracy: 0.9167 - val_loss: 0.2393
Epoch 6/15
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - accuracy: 0.9831 - loss: 0.0969 - val_accuracy: 0.9417 - val_loss: 0.1648
Epoch 7/15
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 72ms/step - accuracy: 0.9957 - loss: 0.0445 - val_accuracy: 0.9729 - val_loss: 0.0907
Epoch 8/15
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 73ms/step - accuracy: 0.9942 - loss: 0.0338 - val_accuracy: 0.9812 - val_loss: 0.

c:\Users\murza\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


64/64 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - accuracy: 0.2651 - loss: 1.6025 - val_accuracy: 0.5861 - val_loss: 0.9297
Epoch 2/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - accuracy: 0.6517 - loss: 0.7778 - val_accuracy: 0.6694 - val_loss: 0.6917
Epoch 3/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - accuracy: 0.7658 - loss: 0.5706 - val_accuracy: 0.7639 - val_loss: 0.5713
Epoch 4/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 6s 98ms/step - accuracy: 0.8417 - loss: 0.4083 - val_accuracy: 0.7889 - val_loss: 0.4926
Epoch 5/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 6s 89ms/step - accuracy: 0.8996 - loss: 0.3025 - val_accuracy: 0.7194 - val_loss: 0.5848
Epoch 6/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - accuracy: 0.9240 - loss: 0.2318 - val_accuracy: 0.8444 - val_loss: 0.3793
Epoch 7/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 6s 95ms/step - accuracy: 0.9617 - loss: 0.1567 - val_accuracy: 0.8917 - val_loss: 0.3036
Epoch 8/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - accuracy: 0.9757 - loss: 0.1171 - val_accuracy: 0.8806 - val_loss: 0.

...

43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 82ms/step - accuracy: 0.2660 - loss: 1.7124 - val_accuracy: 0.2542 - val_loss: 1.3583
Epoch 2/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - accuracy: 0.3443 - loss: 1.2916 - val_accuracy: 0.3750 - val_loss: 1.2839
Epoch 3/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - accuracy: 0.6313 - loss: 0.9500 - val_accuracy: 0.6167 - val_loss: 0.9878
Epoch 4/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 5s 105ms/step - accuracy: 0.8135 - loss: 0.5037 - val_accuracy: 0.6708 - val_loss: 0.8702
Epoch 5/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 5s 107ms/step - accuracy: 0.9298 - loss: 0.2160 - val_accuracy: 0.6875 - val_loss: 0.9762
Epoch 6/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - accuracy: 0.9745 - loss: 0.0933 - val_accuracy: 0.7042 - val_loss: 0.9655
Epoch 7/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 82ms/step - accuracy: 0.9890 - loss: 0.0524 - val_accuracy: 0.6958 - val_loss: 0.9792
Epoch 8/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 92ms/step - accuracy: 0.9974 - loss: 0.0224 - val_accuracy: 0.7667 - val_loss: 

c:\Users\murza\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - accuracy: 0.2404 - loss: 2.0912 - val_accuracy: 0.2500 - val_loss: 1.3669
Epoch 2/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.2871 - loss: 1.3358 - val_accuracy: 0.2500 - val_loss: 1.3433
Epoch 3/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.4726 - loss: 1.1120 - val_accuracy: 0.3167 - val_loss: 1.4342
Epoch 4/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 110ms/step - accuracy: 0.7544 - loss: 0.7329 - val_accuracy: 0.4333 - val_loss: 1.4823
Epoch 5/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.8976 - loss: 0.4055 - val_accuracy: 0.5000 - val_loss: 1.8461
Epoch 6/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - accuracy: 0.9377 - loss: 0.1886 - val_accuracy: 0.4333 - val_loss: 2.1826
Epoch 7/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - accuracy: 0.9814 - loss: 0.1111 - val_accuracy: 0.4667 - val_loss: 2.3195
Epoch 8/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.9935 - loss: 0.0611 - val_accuracy: 0.4833 - val_los

c:\Users\murza\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 146ms/step - accuracy: 0.2720 - loss: 3.1481 - val_accuracy: 0.2833 - val_loss: 1.3887
Epoch 2/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - accuracy: 0.3055 - loss: 1.3773 - val_accuracy: 0.2500 - val_loss: 1.3809
Epoch 3/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.3797 - loss: 1.3549 - val_accuracy: 0.2667 - val_loss: 1.3588
Epoch 4/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.4923 - loss: 1.2703 - val_accuracy: 0.3833 - val_loss: 1.3246
Epoch 5/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - accuracy: 0.7095 - loss: 1.0222 - val_accuracy: 0.4500 - val_loss: 1.2643
Epoch 6/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.8115 - loss: 0.7140 - val_accuracy: 0.4333 - val_loss: 1.3558
Epoch 7/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.8865 - loss: 0.4375 - val_accuracy: 0.5000 - val_loss: 1.3322
Epoch 8/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.9317 - loss: 0.2659 - val_accuracy: 0.4000 - val_l